In [48]:
from math import sqrt
import pandas as pd
import tensorflow as tf
import numpy as np
import datetime
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [32]:
dataset = pd.read_csv('dataset.csv')
dataset.set_index(['date','sex','age','height','timezone'], inplace=True)
dataset.head()
#dataset.shape

,,,,,steps,bedin,bedout,sleep duration,deepduration,lightduration
date,sex,age,height,timezone,,,,,,
2016-04-02,1.0,39.0,174.0,1.0,2547.55,23.78,7.82,7.76,3.24,4.49
2016-04-03,1.0,39.0,174.0,1.0,329.53,23.41,6.48,6.93,2.54,4.38
2016-04-05,1.0,39.0,174.0,1.0,9031.89,22.95,6.73,7.19,3.04,4.14
2016-04-06,1.0,39.0,174.0,1.0,15331.70,22.44,6.18,7.61,3.51,4.14
2016-04-07,1.0,39.0,174.0,1.0,5222.45,24.57,4.04,3.38,1.92,1.49


In [36]:
# Normalizzazione
df_training = dataset.astype(float)

scaler = MinMaxScaler()
scaler = scaler.fit(df_training)
df_training_scaled = scaler.transform(df_training)

In [37]:
print(df_training_scaled)

[[0.01500376 0.90356605 0.91388359 0.03490621 0.01494396 0.06262204]
 [0.00193563 0.90201741 0.90749046 0.03117269 0.01171533 0.06108787]
 [0.05320818 0.90009208 0.90868321 0.03234222 0.01402149 0.05774059]
 ...
 [0.04211869 0.90461242 0.91221374 0.03126265 0.01005489 0.06610879]
 [0.05310389 0.90913276 0.91173664 0.02716927 0.01023938 0.0525802 ]
 [0.04125955 0.90888163 0.91183206 0.02730421 0.01540519 0.03751743]]


In [41]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x_data = x_scaler.fit_transform(dataset[['steps','bedin','bedout','sleep duration','deepduration','lightduration']])
y_data = y_scaler.fit_transform(dataset[['sleep duration']])

[[0.03490621]
 [0.03117269]
 [0.03234222]
 ...
 [0.03126265]
 [0.02716927]
 [0.02730421]]


In [42]:
def custom_ts_multi_data_prep(dataset, target, start, end, window, horizon):
    X = []
    y = []
    start = start + window
    if end is None:
        end = len(dataset) - horizon

    for i in range(start, end):
        indices = range(i-window, i)
        X.append(dataset[indices])

        indicey = range(i+1, i+1+horizon)
        y.append(target[indicey])
    return np.array(X), np.array(y)

In [46]:
hist_window = 1
horizon = 1
TRAIN_SPLIT = 30000
x_train, y_train = custom_ts_multi_data_prep(x_data, y_data, 0, TRAIN_SPLIT, hist_window, horizon)
x_vali, y_vali = custom_ts_multi_data_prep(x_data, y_data, TRAIN_SPLIT, None, hist_window, horizon)

In [49]:
batch_size = 256
buffer_size = 150

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(buffer_size).batch(batch_size).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_vali, y_vali))
val_data = val_data.batch(batch_size).repeat()

In [50]:
lstm_model = tf.keras.models.Sequential([
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True), 
                               input_shape=x_train.shape[-2:]),
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(units=horizon),
])
lstm_model.compile(optimizer='adam', loss='mse')


In [51]:
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 1, 400)           331200    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1, 20)             8020      
                                                                 
 bidirectional_1 (Bidirectio  (None, 300)              205200    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 20)                6020      
                                                                 
 dense_3 (Dense)             (None, 20)                420       
                                                                 
 dropout (Dropout)           (None, 20)               

In [ ]:
history = lstm_model.fit(train_data,epochs=150,steps_per_epoch=100,validation_data=val_data,validation_steps=50,verbose=1)